In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
from sqlalchemy import create_engine

import os
import csv
import re
from datetime import datetime

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [8]:
loc = '/data1/SO_predict_DATA/'
db_file = 'test.db'
r_db_name = db_file
w_db_name = 'C_test.db'

In [3]:
t = pd.read_csv(loc+'Test.csv', nrows = 5)

In [4]:
t

,Id,Title,Body
0,6034196,Getting rid of site-specific hotkeys,<p>How do I disable site-specific hotkeys if (...
1,6034197,Nodes inside Cisco VPN. Incoming SSH requests ...,<p>I've a gateway-to-gateway VPN setup between...
2,6034198,Remove old vCenter servers from VMWare vSphere...,<p>After changing our vCenter servers recently...
3,6034199,Replace <span> element with var containing html,"<p>I have a variable i lifted the contents of,..."
4,6034200,Will PHP included html content affect my seo?,<p>Today i purchase a small CMS system. With t...


In [6]:
if not os.path.isfile(loc+db_name):
    start = datetime.now()
    disk_engine = create_engine('sqlite:///'+loc+db_name)
    #start = dt.datetime.now()
    chunksize = 180000
    j = 0
    index_start = 1
    for df in pd.read_csv(loc+'Test.csv', names=['Id', 'Title', 'Body'], chunksize=chunksize, iterator=True, encoding='utf-8', ):
        df.index += index_start
        j+=1
        print('{} rows'.format(j*chunksize))
        df.to_sql('data', disk_engine, if_exists='append')
        index_start = df.index[-1] + 1
    print("Time taken :", datetime.now() - start)
else:
    print("Already Exists")

180000 rows
360000 rows
540000 rows
720000 rows
900000 rows
1080000 rows
1260000 rows
1440000 rows
1620000 rows
1800000 rows
1980000 rows
2160000 rows
Time taken : 0:01:13.533024


In [7]:
#http://www.sqlitetutorial.net/sqlite-python/create-tables/
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(loc+db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
 
    return None

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print(e)
        
def checkTableExists(dbcon):
    cursr = dbcon.cursor()
    str = "select name from sqlite_master where type='table'"
    table_names = cursr.execute(str)
    print("Tables in the databse:")
    tables =table_names.fetchall() 
    print(tables[0][0])
    return(len(tables))

def create_database_table(database, query):
    conn = create_connection('/'+database)
    if conn is not None:
        print("connctn sucess")
        create_table(conn, query)
        checkTableExists(conn)
    else:
        print("Error! cannot create the database connection.")
    conn.close()

sql_create_table = """CREATE TABLE IF NOT EXISTS clean_test (title text NOT NULL,body text NOT NULL, code text, words_pre integer, words_post integer, is_code integer);"""
create_database_table("C_test.db", sql_create_table)

connctn sucess
Tables in the databse:
clean_test


In [13]:
# Don't Run cleans DB

start = datetime.now()
read_db = r_db_name
write_db = w_db_name
if os.path.isfile(loc+read_db):
    conn_r = create_connection(read_db)
    if conn_r is not None:
        reader =conn_r.cursor()
        reader.execute("SELECT Title, Body From data")

if os.path.isfile(loc+write_db):
    conn_w = create_connection(write_db)
    if conn_w is not None:
        tables = checkTableExists(conn_w)
        writer =conn_w.cursor()
        if tables != 0:
            writer.execute("DELETE FROM clean_test WHERE 1")
            print("Cleared All the rows")
print("Time taken to run this cell :", datetime.now() - start)

Tables in the databse:
clean_test
Cleared All the rows
Time taken to run this cell : 0:00:00.002996


## ----------------- Preprocess ------------------------------

In [14]:
def striphtml(data):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(data))
    return cleantext
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

In [15]:
#http://www.bernzilla.com/2008/05/13/selecting-a-random-row-from-an-sqlite-table/

start = datetime.now()
preprocessed_data_list=[]
reader.fetchone()
questions_with_code=0
len_pre=0
len_post=0
questions_proccesed = 0

for row in reader:
    is_code = 0

    title, question = row[0], row[1]

    if '<code>' in question:
        questions_with_code+=1
        is_code = 1
    x = len(question)+len(title)
    len_pre+=x
    
    title=title.encode('utf-8')
    question = question.encode('utf-8')
    title=str(title)
    question=str(question)
    

    code = str(re.findall(r'<code>(.*?)</code>', question, flags=re.DOTALL))

    question=re.sub('<code>(.*?)</code>', '', question, flags=re.MULTILINE|re.DOTALL)
    question=striphtml(question)#.encode('utf-8'))

    #title=title.encode('utf-8')

   # question=str(title)+" "+str(question)
   # question=re.sub(r'[^A-Za-z]+',' ',question)
    title = re.sub(r'[^A-Za-z0-9#+.\-]+',' ',title)
    question = re.sub(r'[^A-Za-z0-9#+.\-]+',' ',question)
    words=word_tokenize(str(question.lower()))

    #Removing all single letter and and stopwords from question exceptt for the letter 'c'
    question=' '.join(str(stemmer.stem(j)) for j in words if j not in stop_words and (len(j)!=1 or j=='c'))
    
    #same for title
    
    words=word_tokenize(str(title.lower()))

    #Removing all single letter and and stopwords from question exceptt for the letter 'c'
    title=' '.join(str(stemmer.stem(j)) for j in words if j not in stop_words and (len(j)!=1 or j=='c'))

    len_post+=len(question)+len(title)
    tup = (title,question,code,x,len(question)+len(title),is_code)
    questions_proccesed += 1
    writer.execute("insert into clean_test(title,body,code,words_pre,words_post,is_code) values (?,?,?,?,?,?)",tup)
    if (questions_proccesed%100000==0):
        print("number of questions completed=",questions_proccesed)

no_dup_avg_len_pre=(len_pre*1.0)/questions_proccesed
no_dup_avg_len_post=(len_post*1.0)/questions_proccesed

print( "Avg. length of questions(Title+Body) before processing: %d"%no_dup_avg_len_pre)
print( "Avg. length of questions(Title+Body) after processing: %d"%no_dup_avg_len_post)
print ("Percent of questions containing code: %d"%((questions_with_code*100.0)/questions_proccesed))

print("Time taken to run this cell :", datetime.now() - start)

conn_r.commit()
conn_w.commit()
conn_r.close()
conn_w.close()

number of questions completed= 100000
number of questions completed= 200000
number of questions completed= 300000
number of questions completed= 400000
number of questions completed= 500000
number of questions completed= 600000
number of questions completed= 700000
number of questions completed= 800000
number of questions completed= 900000
number of questions completed= 1000000
number of questions completed= 1100000
number of questions completed= 1200000
number of questions completed= 1300000
number of questions completed= 1400000
number of questions completed= 1500000
number of questions completed= 1600000
number of questions completed= 1700000
number of questions completed= 1800000
number of questions completed= 1900000
number of questions completed= 2000000
Avg. length of questions(Title+Body) before processing: 1148
Avg. length of questions(Title+Body) after processing: 336
Percent of questions containing code: 56
Time taken to run this cell : 1:08:15.763964


In [10]:
# conn_r.commit()
# conn_w.commit()
# conn_r.close()
# conn_w.close()